# Programing Assignment III: Quantum Computing Primitives

In [2]:
from numpy import pi as pi
from qiskit import *
from my_quantum_circuit import MyQuantumCircuit

In [ ]:
IBMQ_API_TOKEN: str = ''
IBMQ.save_account(IBMQ_API_TOKEN)
IBMQ.load_account()
pass

## Exercise I -  Programming Quantum Arithmetics with Qiskit - Increment and Decrement

Quantum computing allows for completing traditional arithmetic operations exploiting the superposition of states, e.g., we can increment or decrement in parallel quantum states in superposition. Because of the non-cloning theorem, we can't perform assignments, but only quantum state updates, like increment and decrement.

This exercise aims to implement 3-qubit increment and decrement operators, =+ -=, using the IBM Qiskit framework and its Aer quantum computer simulator.

As part of the exercise:
1. Prepare a superposition of quantum states using HAD QPU instructions. You can decide freely which superposition. This will determine the concurrent increment ordecrement operator.
2. Plot the quantum states using the circle notation. IBM Qiskit does not support visualization for this step, you will need to implement a Python routine to visualize the quantum states of our qubit. You can find a Python code to perform this task at 1.1.1 - Python Code for Circle Notation. You also need to check the Qiskit function to retrieve the complex number corresponding to a quantum state of our qubit system so you can then plot it.
3. Perform an increment operation (by 1) +=
4. Plot the quantum states using the circular notation
5. Perform a decrement operation (by 1) -=
6. Plot the quantum states using the circular notation

Answer the following questions:
1. What happens if we have an overflow (for the increment) or underflow (for the decrement)? Show the results of the overflow and underflow with an experiment.
2. Does increment and decrement operation work also with negative integer numbers? Motivate with an example.

In [21]:
class IncDecQuantumCircuit(MyQuantumCircuit):

    def increment_qreg(self, qreg_index: int):
        """
        Quantum Increment operator.
        :param qreg_index: The index of the quantum register to be incremented.
        :return: self
        """
        qreg = self.circuit.qregs[qreg_index]
        self.circuit.ccx(qreg[0], qreg[1], qreg[2])
        self.circuit.cx(qreg[0], qreg[1])
        self.circuit.x(qreg[0])
        return self


    def decrement_qreg(self, qreg_index: int):
        """
        Quantum Decrement operator.
        :param qreg_index: The index of the quantum register to be decremented.
        :return: self
        """
        qreg = self.circuit.qregs[qreg_index]
        self.circuit.x(qreg[0])
        self.circuit.cx(qreg[0], qreg[1])
        self.circuit.ccx(qreg[0], qreg[1], qreg[2])
        return self


class QFTQuantumCircuit(MyQuantumCircuit):

    def quantum_fourier_transform_qreg(self, qreg_index: int):
        """
        Quantum Fourier Transform.
        :param qreg_index: The index of the quantum register on which we want to apply the QFT.
        :return: If the target quantum register is an integer number, return a discrete sinusoidal wave of frequency equal to this number. Otherwise, if the register is a sine wave, transform it to its corresponding frequency. Frequency is the number of oscillations per total number of states in the target register.
        """
        circuit = self.circuit
        qreg = circuit.qregs[qreg_index]
        qreg_size = int(qreg.size)

        # Apply HADs and CPHASEs:
        for qft_block in range(qreg_size - 1, -1, -1):
            circuit.h(qreg[qft_block])
            for j in range(qft_block - 1, -1, -1):
                theta = - pi / 2 ** (qft_block - j)  # Beware: rotation angle theta must be negative!

                circuit.cp(
                    theta=theta,
                    control_qubit=qreg[j],
                    target_qubit=qreg[qft_block]
                )

            circuit.barrier()

        # Apply SWAPs:
        for k in range(qreg_size // 2):
            circuit.swap(qreg[k], qreg[qreg_size - k - 1])

        return self

    def inverse_quantum_fourier_transform_qreg(self, qreg_index: int):
        circuit = self.circuit
        qreg = circuit.qregs[qreg_index]
        qreg_size = int(qreg.size)
        # Apply SWAPs:
        for k in range(qreg_size // 2):
            circuit.swap(qreg[k], qreg[qreg_size - k - 1])

        # Apply HADs and CPHASEs:
        for qft_block in range(qreg_size):
            circuit.barrier()
            circuit.h(qreg[qft_block])
            for j in range(qft_block + 1, qreg_size):
                theta = pi / 2 ** (j - qft_block)
                circuit.cp(theta=theta, target_qubit=qreg[j], control_qubit=qreg[qft_block])

        return self

    def generate_signals(self, qreg_index: int, qubit_index_theta_dict: dict[int, float]):
        qreg = self.circuit.qregs[qreg_index]
        for qubit_index in qubit_index_theta_dict:
            phase = qubit_index_theta_dict[qubit_index]
            self.circuit.p(theta=phase, qubit=qreg[qubit_index])
        return self

In [ ]:
# Increment operation on a 3-qubit register:
N_QUBITS = 3
INITIAL_VALUE = 3
HAD_GATE_INDICES = [2]

MY_CIRCUIT = IncDecQuantumCircuit()

RESULTS = MY_CIRCUIT\
    .create_simple_circuit(n_qubits=N_QUBITS, name='Increment Circuit')\
    .prepare_qreg_step(qreg_index=0,init_val=INITIAL_VALUE,had_gate_indices=HAD_GATE_INDICES)\
    .save_statevector(statevector_label='pre-increment')\
    .increment_qreg(qreg_index=0)\
    .save_statevector(statevector_label='post-increment')\
    .measure_qreg_to_creg(qreg_index=0, creg_index=0)\
    .execute_circuit()\
    .plot_statevector(statevector_label='pre-increment', useTwosComplement=False)\
    .plot_statevector(statevector_label='post-increment', useTwosComplement=False)\
    .plot_results()[1]

RESULTS

In [ ]:
MY_CIRCUIT.circuit.draw('mpl')

In [ ]:
# Decrement operation on a 3-qubit register:
N_QUBITS = 3
INITIAL_VALUE = 0
HAD_GATE_INDICES = None # [2]

MY_CIRCUIT = IncDecQuantumCircuit()

RESULTS = MY_CIRCUIT\
    .create_simple_circuit(n_qubits=N_QUBITS,name='Decrement Circuit')\
    .prepare_qreg_step(qreg_index=0,init_val=INITIAL_VALUE,had_gate_indices=HAD_GATE_INDICES)\
    .save_statevector(statevector_label='pre-decrement')\
    .decrement_qreg(qreg_index=0)\
    .save_statevector(statevector_label='post-decrement')\
    .measure_qreg_to_creg(qreg_index=0,creg_index=0)\
    .execute_circuit()\
    .plot_statevector(statevector_label='pre-decrement', useTwosComplement=False)\
    .plot_statevector(statevector_label='post-decrement', useTwosComplement=False)\
    .plot_results()[1]

print('Resulted measurements:')
RESULTS

In [ ]:
MY_CIRCUIT.circuit.draw('mpl')

## Exercise II -  Programming the Quantum Fourier Transform (QFT) with Qiskit

The quantum Fourier transform is a crucial building block of many quantum algorithms, from Shor’s factoring algorithm to quantum phase estimation and simulations. As we saw in the lectures, QFT, like the conventional DFT and FFT, can be used to transform a function from the time domain into the frequency domain. Information about the periodic behavior of a function can be, for instance, encoded in the phases of the quantum states.

The goal of this exercise is to compute a 3-qubit QFT of \[0, 1, 0, 0, 0, 0, 0, 0\] using IBM Qiskit and execute it on the Qiskit Aer quantum computer simulator.  As part of this task, develop a three-qubit QFT from scratch using Qiskit, following the circuit formulation in the lecture. Note that the IBM Qiskit provide a QFT function without the need to implement it from scratch. However, the Qiskit QFT code has a different ordering, as explained in the textbook, as pointed out in this GitHub issue: https://github.com/Qiskit/qiskit/pull/4849.

As part of the exercise:
1. Prepare the input \[0, 1, 0, 0, 0, 0, 0, 0\].
2. Plot the quantum states using the circle notation
3. Perform the QFT
4. Plot the quantum states using the circle notation

In [ ]:
# Example of QFT applied on a specified frequency for the sinusoidal wave:
INITIAL_VALUE = 1
HAD_GATE_INDICES = None

MY_CIRCUIT = QFTQuantumCircuit()

RESULTS = MY_CIRCUIT\
    .create_simple_circuit(n_qubits=4)\
    .prepare_qreg_step(qreg_index=0,init_val=INITIAL_VALUE,had_gate_indices=HAD_GATE_INDICES)\
    .save_statevector(statevector_label='pre-QFT')\
    .quantum_fourier_transform_qreg(qreg_index=0)\
    .save_statevector(statevector_label='post-QFT')\
    .measure_qreg_to_creg(qreg_index=0, creg_index=0)\
    .execute_circuit(n_shots=10000)\
    .plot_statevector(statevector_label='pre-QFT')\
    .plot_statevector(statevector_label='post-QFT')\
    .plot_results()[1]

RESULTS

# Applying QFT 1 time yields 2.
# Applying QFT 2 times yields a similar signal pattern but phase rotations occur in the opposite direction.
# Applying QFT 3 times yields 14 = 16 - 2.
# Applying QFT 4 times yields the initial signals.

In [ ]:
# QFT applied on discrete sinusoidal wave example (with frequency = 1):
INITIAL_VALUE = None
HAD_GATE_INDICES = [0, 1, 2, 3]

THETAS = {
    0: pi/8,
    1: pi/4,
    2: pi/2,
    3: pi,
}

MY_CIRCUIT = QFTQuantumCircuit()

RESULTS = MY_CIRCUIT\
    .create_simple_circuit(4)\
    .prepare_qreg_step(qreg_index=0,init_val=INITIAL_VALUE,had_gate_indices=HAD_GATE_INDICES)\
    .generate_signals(qreg_index=0, qubit_index_theta_dict=THETAS)\
    .save_statevector(statevector_label='pre-QFT')\
    .quantum_fourier_transform_qreg(qreg_index=0)\
    .save_statevector(statevector_label='post-QFT')\
    .measure_qreg_to_creg(qreg_index=0, creg_index=0)\
    .execute_circuit(n_shots=10000)\
    .plot_statevector(statevector_label='pre-QFT')\
    .plot_statevector(statevector_label='post-QFT')\
    .plot_results()[1]

RESULTS

# Applying QFT 1 time yields 2.
# Applying QFT 2 times yields a similar signal pattern but phase rotations occur in the opposite direction.
# Applying QFT 3 times yields 14 = 16 - 2.
# Applying QFT 4 times yields the initial signals.

In [ ]:
MY_CIRCUIT.circuit.draw('mpl')

## Assignment Report

In your report, please provide:
1. Briefly describe the quantum circuit design, including the gates used and their sequence.
2. Code implementation of the quantum circuit using Qiskit.
3. Simulation results
4. Brief discussion of the results.